<a href="https://colab.research.google.com/github/waswapeter/CEE_Class-Codes-Data/blob/main/farm_planning/farm_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Farm Planning

## Objective and Prerequisites

Cultivate your modeling skills with this example, where you’ll learn how to solve a complex, multi-period production planning problem that involves optimizing the operations of a farm over five years.

More information on this type of model can be found in example #8 of the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 262 – 263 and 312 – 315.

This modeling example is at the advanced level, where we assume that you know Python and the Gurobi Python API and that you have advanced knowledge of building mathematical optimization models. Typically, the objective function and/or constraints of these examples are complex or require advanced features of the Gurobi Python API.

**Download the Repository** <br /> You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).

---
## Problem Description

A farmer with a 200-acre farm wishes to create a five-year production plan.

Currently, he has a herd of 120 cows comprising 20 heifers (young female cows) and 100 adult dairy cows. Each heifer requires 2/3 of an acre to support it and each dairy cow requires one acre.

On average, a dairy cow produces 1.1 calves per year. Half of these calves will be bullocks that are sold shortly after birth for an average of $\$30$ each. The remaining calves, heifers, can either be sold for $\$40$ or raised until age two when they will become dairy cows. For the current year, all heifers identified for sale have already been sold.

The general practice is to sell all dairy cows at the age of 12 for an average of $\$120$ each. However, each year an average of $5\%$ of heifers and $2\%$ of dairy cows die. At present, the farmer’s herd of 120 cows is evenly distributed with 10 cows per age from newborn to 11 years old.

The milk from a dairy cow can be sold for $\$370$ annually. The farmer can currently house up to 130 cows, but this capacity limit can be increased for $\$200$ per additional cow.

Each dairy cow requires 0.6 tons of grain and 0.7 tons of sugar beet per year. Both of these can be grown on the farm. Each acre can yield 1.5 tons of sugar beet. However, only 80 acres are suitable for growing grain, and those acres have different levels of productivity as follows:

| Land Group | Area (Acres) | Grain Production (Tons/Acre) |
| --- | --- | --- |
| Group 1 | 20 | 1.10 |
| Group 2 | 30 | 0.90 |
| Group 3 | 20 | 0.80 |
| Group 4 | 10 | 0.65 |

Sugar beet can be bought for $\$70$ a ton and sold for $\$58$ a ton. Grain can be bought for $\$90$ a ton and sold for $\$75$ a ton.

The annual labor requirements for cows as well as grain and sugar beet production are as follows:

| <i></i> | Labor Required (Hr/Year) |
| --- | --- |
| Heifer | 10 |
| Milk-producing Cow | 42 |
| Acre Devoted to Grain | 4 |
| Acre Devoted to Sugar Beet| 14 |

Other annual costs are as follows:

| <i></i> | Cost (USD/Year) |
| --- | --- |
| Heifer | 50 |
| Milk-producing Cow | 100 |
| Acre Devoted to Grain | 15 |
| Acre Devoted to Sugar Beet| 10 |

Labor currently costs the farmer $\$4000$ per year and that cost provides 5,500 hours of labor. Additional labor can be paid for at the rate of $\$1.20$ per hour.

Any capital expenditure can be financed with a 10-year loan at $15\%$ interest annually. The interest and principal are paid back in 10 equal annual payments. It is prohibited for the annual cash flow to be negative.

The farmer does not want to reduce the total number of dairy cows at the end of the five-year period by more than $50\%$, nor does he want to increase their number by more than $75\%$.

What plan should the farmer follow over the next five years in order to maximize profit?

---
## Model Formulation

### Sets and Indices

$t \in \text{Years}=\{1,2,\dots,5\}$: Set of years.

$l \in \text{Lands}=\{1,2,3,4\}$: Set of land groups.

$k \in \text{Ages}=\{1,2,\dots,12\}$: Set of cow ages.

### Parameters

$\text{Installment} \in \mathbb{R}^+$: Annual payment for each $\$200$ loan. It can be computed as follows.

$$
\text{Installment}= \text{Loan}*r*\frac{(1+r)^n}{(1+r)^n-1},
$$

where $\text{Loan}$ is the loan amount of $\$200$, $r$ is the interest rate of $15\%$, and $n$ represents the ten-year period. Note that the book reports a value of $\$39.71$ instead of $\$39.85$.

$\text{Housing_cap} \in \mathbb{N}$: Number of cows that can be housed.

$\text{Land_cap} \in \mathbb{R}^+$: Land available (in Acres).

$\text{Labor_cap} \in \mathbb{R}^+$: Regular labor available (in hours) in a year.

$\text{GR_intake} \in \mathbb{R}^+$: Tons of grain consumed by a dairy cow in a year.

$\text{SB_intake} \in \mathbb{R}^+$: Tons of sugar beet consumed by a dairy cow in a year.

$\text{HF_land} \in \mathbb{R}^+$: Acres required for sustaining each heifer.

$\text{HF_labor} \in \mathbb{R}^+$: Hours of labor required in a year by each heifer.

$\text{Cow_labor} \in \mathbb{R}^+$: Hours of labor required in a year by each dairy cow.

$\text{GR_labor} \in \mathbb{R}^+$: Hours of labor required in a year by each acre of land devoted to grains.

$\text{SB_labor} \in \mathbb{R}^+$: Hours of labor required in a year by each acre of land devoted to sugar beet.

$\text{HF_decay} \in [0,1] \subset \mathbb{R}^+$: Average proportion of heifers that die in a year.

$\text{Cow_decay} \in [0,1] \subset \mathbb{R}^+$: Average proportion of dairy cows that die in a year.

$\text{Birthrate} \in \mathbb{R}^+$: Expected number of calves produced by a dairy cow in a year.

$\text{Min_final_cows} \in \mathbb{N}$: Minimum number of dairy cows at the end of the planning horizon.

$\text{Max_final_cows} \in \mathbb{N}$: Maximum number of dairy cows at the end of the planning horizon.

$\text{Initial_HF} \in \mathbb{R}^+$: Number of heifers of each age at the beginning of the planning horizon.

$\text{Initial_cows} \in \mathbb{R}^+$: Number of dairy cows of each age at the beginning of the planning horizon.

$\text{BL_price} \in \mathbb{R}^+$: Price for selling one bullock.

$\text{HF_price} \in \mathbb{R}^+$: Price for selling one heifer.

$\text{Cow_price} \in \mathbb{R}^+$: Price for selling one dairy cow.

$\text{Milk_price} \in \mathbb{R}^+$: Price for selling the milk produced by a dairy cow in a year.

$\text{GR_price} \in \mathbb{R}^+$: Price for selling a ton of grain.

$\text{SB_price} \in \mathbb{R}^+$: Price for selling a ton of sugar beet.

$\text{GR_cost} \in \mathbb{R}^+$: Cost for purchasing a ton of grain.

$\text{SB_cost} \in \mathbb{R}^+$: Cost for purchasing a ton of sugar beet.

$\text{Overtime_cost} \in \mathbb{R}^+$: Cost for getting an hour of overtime.

$\text{Regular_time_cost} \in \mathbb{R}^+$: Cost for having 5,500 hours of labor in regular time.

$\text{HF_cost} \in \mathbb{R}^+$: Yearly cost for supporting a heifer.

$\text{Cow_cost} \in \mathbb{R}^+$: Yearly cost for supporting a dairy cow.

$\text{GR_land_cost} \in \mathbb{R}^+$: Yearly cost for supporting an acre of land devoted to grain.

$\text{SB_land_cost} \in \mathbb{R}^+$: Yearly cost for supporting an acre of land devoted to sugar beet.

$\text{SB_yield} \in \mathbb{R}^+$: Sugar beet yield.

$\text{GR_yield}_l \in \mathbb{R}^+$: Grain yield at land group $l$.

$\text{GR_area}_l \in \mathbb{R}^+$: Number of acres suitable for growing grain in land group $l$.

### Decision Variables

$\text{Outlay}_t \in \mathbb{R}^+$: Amount of money (in units of $\$200$) spent on renting houses in year $t$.

$\text{Overtime}_t \in \mathbb{R}^+$: Number of extra labor hours needed in year $t$.

$\text{Newborn}_t \in \mathbb{R}^+$: Number of newborn heifers left in year $t$ to be raised.

$\text{HF_sell}_t \in \mathbb{R}^+$: Number of newborn heifers to sell in year $t$.

$\text{Profit}_t \in \mathbb{R}^+$: Profit attained in year $t$.

$\text{SB_buy}_t \in \mathbb{R}^+$: Number of tons of sugar beet to buy in year $t$.

$\text{SB_sell}_t \in \mathbb{R}^+$: Number of tons of sugar beet to sell in year $t$.

$\text{GR_buy}_t \in \mathbb{R}^+$: Number of tons of grain to buy in year $t$.

$\text{GR_sell}_t \in \mathbb{R}^+$: Number of tons of grain to sell in year $t$.

$\text{SB}_t \in \mathbb{R}^+$: Number of tons of sugar beet to grow in year $t$.

$\text{GR}_{t,l} \in \mathbb{R}^+$: Number of tons of grain to grow in year $t$ at land group $l$.

$\text{Cows}_{t,k} \in \mathbb{R}^+$: Number of cows of age $k$ available in year $t$.


### Objective Function

- **Profit**: Maximize the total profit (in USD) of the planning horizon. Notice that, to make capital expenditure as costly in latter years as in former ones, it is necessary to subtract pending loan payments.

\begin{equation}
\text{Maximize} \quad Z = \sum_{t \in \text{Years}}{\text{Profit}_t - installment*(t+4)*\text{Outlay}_t}
\end{equation}

### Constraints

- **Housing Capacity**: Livestock cannot exceed the installed capacity plus house rental in year $t$.

\begin{equation}
\text{Newborn}_t + \sum_{k \in \text{Ages} \setminus \{12\}}{\text{Cows}_{t,k}} \leq \text{Housing_cap} + \sum_{d \in \text{Years}: d \leq t}{\text{Outlay}_d} \quad \forall t \in \text{Years}
\end{equation}

- **Food Consumption**: There must be enough food to feed the livestock in year $t$.

- Grain.

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{GR_intake}*\text{Cows}_{t,k}} \leq \text{GR_buy}_t - \text{GR_sell}_t + \sum_{l \in \text{Lands}}{\text{GR}_{t,l}} \quad \forall t \in \text{Years}
\end{equation}

- Sugar beet.

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{SB_intake}*\text{Cows}_{t,k}} \leq \text{SB_buy}_t - \text{SB_sell}_t + \text{SB}_t \quad \forall t \in \text{Years}
\end{equation}

- **Grain Growing**: Grain produced on land $l$ cannot exceed the production capacity of year $t$.

\begin{equation}
\text{GR}_{t,l} \leq \text{GR_yield}_l*\text{GR_area}_l \quad \forall (t,l) \in \text{Years} \times \text{Lands}
\end{equation}

- **Land Capacity**: Use of space in year $t$ cannot exceed available land.

\begin{equation}
\frac{1}{\text{SB_yield}}*\text{SB}_t + \text{HF_land}*(\text{Newborn}_t + \text{Cow}_{t,1}) +
\end{equation}

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}} +
\end{equation}

\begin{equation}
\sum_{l \in \text{Lands}}{\frac{\text{GR}_{t,l}}{\text{GR_yield}_l}} \leq \text{Land_cap} \quad \forall t \in \text{Years}
\end{equation}

- **Labor**: Labor required to run the farm in year $t$ cannot exceed contracted time plus overtime.

\begin{equation}
\text{HF_labor}*(\text{Newborn}_t + \text{Cow}_{t,1}) +
\end{equation}

\begin{equation}
\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cow_labor}*\text{Cow}_{t,k}} +
\end{equation}

\begin{equation}
\sum_{l \in \text{Lands}}{\frac{\text{GR_labor}*\text{GR}_{t,l}}{\text{GR_yield}_l}} +
\end{equation}

\begin{equation}
\frac{\text{SB_labor}*\text{SB}_t}{\text{SB_yield}} \leq \text{Labor_cap} + \text{Overtime}_t \qquad \forall t \in \text{Years}
\end{equation}


- **Continuity**: Livestock in year $t$ have to survive the previous year.

\begin{equation}
\text{Cows}_{t,1} = (1-\text{HF_decay})*\text{Newborn}_{t-1} \quad \forall t \in \text{Years} \setminus \{1\}
\end{equation}

\begin{equation}
\text{Cows}_{t,2} = (1-\text{HF_decay})*\text{Cows}_{t-1,1} \quad \forall t \in \text{Years} \setminus \{1\}
\end{equation}

\begin{equation}
\text{Cows}_{t,k+1} = (1-\text{Cow_decay})*\text{Cows}_{t-1,k} \quad \forall (t,k) \in \text{Years} \setminus \{1\} \times \text{Ages} \setminus \{1,12\}
\end{equation}

- **Heifers Birth**: Heifers born in year $t$ depend on the number of dairy cows.

\begin{equation}
\text{Newborn}_t + \text{HF_sell}_t = \sum_{k \in \text{Ages} \setminus \{1,12\}}{\frac{\text{Birthrate}}{2}*\text{Cows}_{t,k}} \quad \forall t \in \text{Years}
\end{equation}

- **Final Dairy Cows**: The number of dairy cows at the end of the planning horizon must be within tolerances.

\begin{equation}
\text{Min_final_cows} \leq \sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{5,k}} \leq \text{Max_final_cows}
\end{equation}

- **Initial Conditions**: Set the number of livestock available at the beginning of the planning horizon.

\begin{equation}
\text{Cows}_{1,1} = \text{Initial_HF}
\end{equation}

\begin{equation}
\text{Cows}_{1,2} = \text{Initial_HF}
\end{equation}

\begin{equation}
\text{Cows}_{1,k} = \text{Initial_cows} \quad \forall k \in \text{Ages} \setminus \{1,2\}
\end{equation}

- **Yearly Profit**: Profit in year $t$ is driven by operations from crops and livestock, after accounting for labor, land and financial costs.

\begin{equation}
\text{Profit}_t = \frac{\text{BL_price}*\text{Birthrate}}{2}*\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}}
+ \text{HF_price}*\text{HF_sell}_t
+ \text{Cow_price}*\text{Cows}_{t,12}
\end{equation}

\begin{equation}
+ \text{Milk_price}*\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}}
\end{equation}

\begin{equation}
+ \text{GR_price}*\text{GR_sell}_t
+ \text{SB_price}*\text{SB_sell}_t
\end{equation}

\begin{equation}
- \text{GR_cost}*\text{GR_buy}_t
- \text{SB_cost}*\text{SB_buy}_t
\end{equation}

\begin{equation}
- \text{Overtime_cost}*\text{Overtime}_t
- \text{Regular_time_cost}
\end{equation}

\begin{equation}
- \text{HF_cost}*(\text{Newborn}_t + \text{Cows}_{t,1})
- \text{Cow_cost}*\sum_{k \in \text{Ages} \setminus \{1,12\}}{\text{Cows}_{t,k}}
\end{equation}

\begin{equation}
- \text{GR_land_cost}*\sum_{l \in \text{Lands}}{\frac{\text{GR}_{t,l}}{\text{GR_yield}_l}}
- \text{SB_land_cost}*\frac{\text{SB}_t}{\text{SB_yield}} \\
- \text{Installment}*\sum_{d \in \text{Years}:d \leq t}{\text{Outlay}_d}
\quad \forall t \in \text{Years}
\end{equation}

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [2]:
%pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 52.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## Input Data
We define all the input data of the model.

In [4]:
# Parameters

years = [1,2,3,4,5]
lands = [1,2,3,4]
ages = [1,2,3,4,5,6,7,8,9,10,11,12]
cow_ages = [2,3,4,5,6,7,8,9,10,11]

gr_area = {1: 20.0, 2: 30.0, 3: 20.0, 4: 10.0}
gr_yield = {1: 1.1,  2: 0.9, 3: 0.8, 4:0.65}
sb_yield = 1.5
housing_cap = 130
gr_intake = 0.6
sb_intake = 0.7
hf_land = 2/3.0
land_cap = 200
hf_labor = 10/100.0
cow_labor = 42/100.0
gr_labor = 4/100.0
sb_labor = 14/100.0
labor_cap = 5500/100.0
cow_decay = 0.02
hf_decay = 0.05
initial_hf = 9.5
initial_cows = 9.8
birthrate = 1.1
min_final_cows = 50
max_final_cows = 175
bl_price = 30
hf_price = 40
cow_price = 120
milk_price = 370
gr_price = 75
sb_price = 58
gr_cost = 90
sb_cost = 70
overtime_cost = 120
regular_time_cost = 4000
hf_cost = 50
cow_cost = 100
gr_land_cost = 15
sb_land_cost = 10
installment = 39.71

## Model Deployment

We create a model and the variables. For each, year we have continuous variables, which tell us how much sugar beet is grown (in tons), how much grain is bought (in tons), how much grain is sold (in tons), how much sugar beet is bought (in tons), how much sugar beet is sold (in tons), how much extra labor is recruited, how much capital outlay there is, how many heifers are sold at birth, how much profit there is, and how many cows are newborns (age 0).

For each year and each land group, there is a continuous variable, which tells us how much grain is grown on that land group. For each year and each cow age, there is a continuous variable that tells us how many cows exists in the current year of that age.

In [5]:
model = gp.Model('Farming')
sb = model.addVars(years, vtype=GRB.CONTINUOUS, name="SB")
gr_buy = model.addVars(years, vtype=GRB.CONTINUOUS, name="GR_buy")
gr_sell = model.addVars(years, vtype=GRB.CONTINUOUS, name="GR_sell")
sb_buy = model.addVars(years, vtype=GRB.CONTINUOUS, name="SB_buy")
sb_sell = model.addVars(years, vtype=GRB.CONTINUOUS, name="SB_sell")
overtime = model.addVars(years, vtype=GRB.CONTINUOUS, name="Overtime")
outlay = model.addVars(years, vtype=GRB.CONTINUOUS, name="Outlay")
hf_sell = model.addVars(years, vtype=GRB.CONTINUOUS, name="HF_sell")
newborn = model.addVars(years, vtype=GRB.CONTINUOUS, name="Newborn")
profit = model.addVars(years, vtype=GRB.CONTINUOUS, name="Profit")
gr = model.addVars(years, lands, vtype=GRB.CONTINUOUS, name="GR")
cows = model.addVars(years, ages, vtype=GRB.CONTINUOUS, name="Cows")

Restricted license - for non-production use only - expires 2026-11-23


Next we insert the constraints:

There is only a housing capacity for 130 cows per year. It is possible that there are more than 130 cows, but housing these additional cows will require additional costs related to renting houses, which is covered with the `Outlay` variables.

In [6]:
# 1. Housing capacity

HousingCap = model.addConstrs((newborn[year] +
                    cows[year,1] +
                    gp.quicksum(cows[year,age] for age in cow_ages) -
                    gp.quicksum(outlay[d] for d in years if d <= year)
                    <= housing_cap for year in years), name="Housing_cap")

After selling, buying, and producing grains and sugar beet, there needs to be enough of them in storage to feed all the cows.

In [7]:
# 2.1 Food consumption (Grain)

GrainConsumption = model.addConstrs((gp.quicksum(gr_intake*cows[year, age] for age in cow_ages)
                  <= gr_buy[year] - gr_sell[year] + gr.sum(year, '*')
                  for year in years), name="Grain_consumption")

# 2.1 Food consumption (Sugar beet)
SugarbeetConsumption = model.addConstrs((gp.quicksum(sb_intake*cows[year, age] for age in cow_ages)
                  <= sb_buy[year] - sb_sell[year] + sb[year]
                  for year in years), name="Sugar_beet_consumption")

The grain produced on a land group cannot exceed the specified production capacity of that land group.

In [8]:
# 3. Grain growing

GrainGrowing = model.addConstrs((gr[year, land] <= gr_yield[land]*gr_area[land]
                  for year in years for land in lands), name="Grain_growing")

Each cow needs a certain number of acres to support it. The amount depends on the age of the cow. There are 200 acres available at most.

In [9]:
# 4. Land capacity

LandCap = model.addConstrs((sb[year]/sb_yield + hf_land*(newborn[year] + cows[year,1])
                  + gp.quicksum((1/gr_yield[land])*gr[year, land] for land in lands)
                  + gp.quicksum(cows[year, age] for age in cow_ages)
                  <= land_cap for year in years), name="Land_capacity")

Each cow and each acre requires a certain amount of worker time to maintain. The farm is currently able to provide a fixed number of worker hours in a year. Any additional work that needs to be done requires external workers, for which there will be an additional cost.

In [10]:
# 5. Labor

Labor = model.addConstrs((hf_labor*(newborn[year] + cows[year,1])
                  + gp.quicksum(cow_labor*cows[year, age] for age in cow_ages)
                  + gp.quicksum(gr_labor/gr_yield[land]*gr[year,land] for land in lands)
                  + sb_labor/sb_yield*sb[year]
                  <= labor_cap + overtime[year] for year in years), name="Labor")

Each year a certain percentage of the cows die, depending on their age.

In [11]:
# 6.1 Continuity

Continuity1 = model.addConstrs((cows[year,1] == (1-hf_decay)*newborn[year-1]
                  for year in years if year > min(years)),
                 name="Continuity_a")

# 6.2 Continuity

Continuity2 = model.addConstrs((cows[year,2] == (1-hf_decay)*cows[year-1,1]
                  for year in years if year > min(years)),
                 name="Continuity_b")

# 6.3 Continuity

Continuity3 = model.addConstrs((cows[year,age+1] == (1-cow_decay)*cows[year-1,age]
                  for year in years for age in cow_ages if year > min(years)),
                 name="Continuity_c")

Keep track of the number of cows; cows can come into/out of the model by being bought or sold or through birth.

In [12]:
# 7. Heifers birth

HeifersBirth = model.addConstrs((newborn[year] + hf_sell[year]
                  == gp.quicksum(birthrate/2*cows[year,age] for age in cow_ages) for year in years)
                 , name="Heifers_birth")

At the end of the five years, the farmer wants at least 50 and at most 175 diary cows.

In [13]:
# 8. Final dairy cows
FinalDairyCows = model.addRange(gp.quicksum(cows[max(years), age] for age in cow_ages), min_final_cows, max_final_cows, name="Final_dairy_cows" )

In the first year, there are 9.5 one-year-old cows and 9.5 two-year-old cows. In addition, there are 9.8 cows for each age from three to 12. Note that we are solving this as an LP model to make it easier to solve. This can lead to fractional values for variables, which are in reality integers.

In [14]:
# 9.1-9.2 Initial conditions

InitialHeifers = model.addConstrs((initial_hf == cows[1, age] for age in ages if age < 3),
                 name="Initial_conditions")

# 9.3 Initial conditions

InitialCows = model.addConstrs((initial_cows == cows[1, age] for age in ages if age >= 3),
                 name="Initial_condition_cows")

The following constraints determine yearly profits. The costs for labor currently total $\$4,000$. The profit is influenced by the selling of bullocks and heifers, selling of 12-year-old-cows, selling of milk, selling of grain, selling of sugar beet, buying of grain, buying of sugar beet, labor costs, heifer costs, dairy cow costs, grain costs, sugar beet costs, and capital costs.

In [15]:
# 10. Yearly profit

YearlyProfit = model.addConstrs((profit[year]
                  == bl_price*birthrate/2*gp.quicksum(cows[year, age] for age in cow_ages)
                  + hf_price*hf_sell[year] + cow_price*cows[year, 12]
                  + milk_price*gp.quicksum(cows[year, age] for age in cow_ages)
                  + gr_price*gr_sell[year] + sb_price*sb_sell[year]
                  - gr_cost*gr_buy[year] - sb_cost*sb_buy[year]
                  - overtime_cost*overtime[year] - regular_time_cost
                  - hf_cost*(newborn[year] + cows[year,1])
                  - cow_cost*gp.quicksum(cows[year, age] for age in cow_ages)
                  - gr_land_cost*gp.quicksum(gr[year, land]/gr_yield[land] for land in lands)
                  - sb_land_cost*sb[year]/sb_yield
                  - installment*gp.quicksum(outlay[d] for d in years if d <= year)
                  for year in years), name="Yearly_profit")

The total profit of the planning horizon consists of the calculated profits minus the loan payments that remain pending:

In [16]:
# 0. Total profit

model.setObjective(gp.quicksum(profit[year] - installment*(year+4)*outlay[year] for year in years), GRB.MAXIMIZE)

Next, we start the optimization and Gurobi finds the optimal solution.

In [17]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 116 rows, 131 columns and 734 nonzeros
Model fingerprint: 0x7b2f9c98
Coefficient statistics:
  Matrix range     [4e-02, 3e+02]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+02, 1e+02]
  RHS range        [6e+00, 4e+03]
Presolve removed 84 rows and 67 columns
Presolve time: 0.01s
Presolved: 32 rows, 64 columns, 252 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1200000e+32   5.000000e+30   5.120000e+02      0s
      22    1.2171917e+05   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.217191729e+05


---
## Analysis

The optimal plan results in a total profit of $\$121,719.17$ over the five-year period the model covers. The detailed plan for each year is as follows.

### Financial Plan

This plan determines the profit and outlay at each period of the planning horizon. For example, the profit in year 1 is $\$21,906.1$ and the outlay is zero dollars.

In [18]:
rows = ["Profit", "Outlay"]
columns = years.copy()
finance_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year in years:
    if (abs(profit[year].x) > 1e-6):
        finance_plan.loc["Profit", year] = np.round(profit[year].x, 1)
    if (abs(outlay[year].x) > 1e-6):
        finance_plan.loc["Outlay", year] = np.round(outlay[year].x, 1)
finance_plan

,1,2,3,4,5
Profit,21906.1,21888.7,25816.1,26825.8,25282.6
Outlay,0.0,0.0,0.0,0.0,0.0


### Plan for Grains
This plan defines the number of tons of grain to grow for each type of land group (rows) at each year of the planning horizon (columns). It also defines the number of tons of grain to buy and sell at each year of the planning horizon. For example, we are going to grow 22 tons of grain at land group 1 during year 2 . In addition, we are going to buy 35.1 tons of grain during year 2.

In [19]:
rows = lands.copy() + ["Buy", "Sell"]
columns = years.copy()
gr_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, land in gr.keys():
    if (abs(gr[year, land].x) > 1e-6):
        gr_plan.loc[land, year] = np.round(gr[year, land].x, 1)
for year in years:
    if (abs(gr_buy[year].x) > 1e-6):
        gr_plan.loc["Buy", year] = np.round(gr_buy[year].x, 1)
    if (abs(gr_sell[year].x) > 1e-6):
        gr_plan.loc["Sell", year] = np.round(gr_sell[year].x, 1)
gr_plan

,1,2,3,4,5
1,22.0,22.0,22.0,22.0,22.0
2,0.0,0.0,2.8,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
Buy,36.6,35.1,37.8,40.1,33.5
Sell,0.0,0.0,0.0,0.0,0.0


### Plan for Sugar Beet
This plan defines the number of tons of sugar beet to grow at each year of the planning horizon (columns). It also defines the number of tons of sugar beet to buy and sell at each year of the planning horizon. For example, we are going to grow 94 tons of sugar beet during year 2 . In addition, we are going to sell 27.4 tons of sugar beet during year 2.

In [20]:
rows = ["Grow", "Buy", "Sell"]
columns = years.copy()
sb_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year in years:
    if (abs(sb[year].x) > 1e-6):
        sb_plan.loc["Grow", year] = np.round(sb[year].x, 1)
    if (abs(sb_buy[year].x) > 1e-6):
        sb_plan.loc["Buy", year] = np.round(sb_buy[year].x, 1)
    if (abs(sb_sell[year].x) > 1e-6):
        sb_plan.loc["Sell", year] = np.round(sb_sell[year].x, 1)
sb_plan

,1,2,3,4,5
Grow,91.1,94.0,97.7,114.6,131.3
Buy,0.0,0.0,0.0,0.0,0.0
Sell,22.8,27.4,24.6,42.1,66.6


### Plan for Heifers

This plan shows number of heifers to sell and raise at each period of the planning horizon. For example, we are going to sell 40.8 heifers and raise 11.6 heifers during year 2.

In [21]:
rows = ["Sell", "Raise"]
columns = years.copy()
livestock_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year in years:
    if (abs(hf_sell[year].x) > 1e-6):
        livestock_plan.loc["Sell", year] = np.round(hf_sell[year].x, 1)
    if (abs(newborn[year].x) > 1e-6):
        livestock_plan.loc["Raise", year] = np.round(newborn[year].x, 1)
livestock_plan

,1,2,3,4,5
Sell,30.9,40.8,57.4,57.0,50.9
Raise,22.8,11.6,0.0,0.0,0.0


---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC